
# Pedestrian Anomaly Detection using MOT17 (Final Submission)

This single notebook performs:
1. MOT17 preprocessing → YOLO format  
2. Training YOLOv5s, YOLOv5m, YOLOv5m (frozen)  
3. Tracking with DeepSORT  
4. Rule-based anomaly detection  
5. Model comparison & conclusions  

⚠️ Use CPU for preprocessing, GPU for training.


## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Setup YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17783, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 17783 (delta 2), reused 1 (delta 1), pack-reused 17776 (from 2)
Receiving objects: 100% (17783/17783), 16.89 MiB | 29.37 MiB/s, done.
Resolving deltas: 100% (12125/12125), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [ ]:
!nvidia-smi


Tue Jan 27 12:04:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 3. MOT17 → YOLO Preprocessing (CPU ONLY)

In [ ]:

import os, shutil, random
from tqdm import tqdm

MOT17_ROOT = '/content/drive/MyDrive/MOT17'
YOLO_ROOT  = '/content/drive/MyDrive/MOT17_YOLO'

IMG_W, IMG_H = 1920, 1080   # MOT17 resolution

def convert_bbox(x, y, w, h, W, H):
    cx = (x + w / 2) / W
    cy = (y + h / 2) / H
    bw = w / W
    bh = h / H
    return cx, cy, bw, bh

# Create YOLO folders
for split in ['train', 'val']:
    os.makedirs(f"{YOLO_ROOT}/{split}/images", exist_ok=True)
    os.makedirs(f"{YOLO_ROOT}/{split}/labels", exist_ok=True)

# 🔹 USE ONLY MOT17/train
for seq in os.listdir(f"{MOT17_ROOT}/train"):
    if not seq.endswith('FRCNN'):
        continue

    img_dir = f"{MOT17_ROOT}/train/{seq}/img1"
    gt_file = f"{MOT17_ROOT}/train/{seq}/gt/gt.txt"

    if not os.path.exists(gt_file):
        continue

    # Load annotations
    ann = {}
    with open(gt_file) as f:
        for line in f:
            frame, _, x, y, w, h, conf, _, _ = line.strip().split(',')
            if int(conf) == 0:
                continue
            ann.setdefault(int(frame), []).append(
                (float(x), float(y), float(w), float(h))
            )

    # List images
    images = sorted([img for img in os.listdir(img_dir) if img.endswith('.jpg')])
    random.shuffle(images)

    split_idx = int(0.8 * len(images))
    train_imgs = images[:split_idx]
    val_imgs   = images[split_idx:]

    def process(img_list, split):
        for img in tqdm(img_list, desc=f"{seq} → {split}"):
            fid = int(''.join(filter(str.isdigit, img)))

            # Copy image
            shutil.copy(
                f"{img_dir}/{img}",
                f"{YOLO_ROOT}/{split}/images/{seq}_{img}"
            )

            # Write label
            label_path = f"{YOLO_ROOT}/{split}/labels/{seq}_{img.replace('.jpg','.txt')}"
            with open(label_path, 'w') as out:
                if fid in ann:
                    for x, y, w, h in ann[fid]:
                        cx, cy, bw, bh = convert_bbox(x, y, w, h, IMG_W, IMG_H)
                        out.write(f"0 {cx} {cy} {bw} {bh}\n")

    process(train_imgs, 'train')
    process(val_imgs, 'val')



MOT17-13-FRCNN → train:  68%|██████▊   | 410/600 [09:53<04:35,  1.45s/it]


KeyboardInterrupt: 

In [ ]:
!ls /content/drive/MyDrive/MOT17_YOLO/train/images | head


MOT17-02-FRCNN_000001.jpg
MOT17-02-FRCNN_000006.jpg
MOT17-02-FRCNN_000007.jpg
MOT17-02-FRCNN_000008.jpg
MOT17-02-FRCNN_000010.jpg
MOT17-02-FRCNN_000011.jpg
MOT17-02-FRCNN_000012.jpg
MOT17-02-FRCNN_000013.jpg
MOT17-02-FRCNN_000015.jpg
MOT17-02-FRCNN_000016.jpg


In [ ]:
!ls /content/drive/MyDrive/MOT17


test  train


## 4. data.yaml

In [ ]:

with open('mot17.yaml','w') as f:
    f.write("""
path: /content/drive/MyDrive/MOT17_YOLO
train: train/images
val: val/images
nc: 1
names: ['person']
""")


## ⚠️ SWITCH RUNTIME TO GPU NOW

## 5. Train YOLOv5s

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data mot17.yaml --weights yolov5s.pt --name yolov5s_mot17

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
      33/49       4.1G    0.04437    0.04946          0        583        640:  24% 47/193 [00:40<02:14,  1.09it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      33/49       4.1G    0.04434    0.04928          0        313        640:  25% 48/193 [00:40<01:48,  1.34it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      33/49       4.1G    0.04437     0.0493          0        432        640:  25% 49/193 [00:42<02:21,  1.02it/s]/content/yolov5/train.py:414: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp

In [ ]:
!cp -r /content/yolov5/runs /content/drive/MyDrive/YOLO_BACKUP_ONGOING


## 6. Train YOLOv5m

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data mot17.yaml --weights yolov5m.pt --name yolov5m_mot17

## 7. Train YOLOv5m (Frozen Backbone)

In [ ]:
!python train.py --img 640 --batch 16 --epochs 50 --data mot17.yaml --weights yolov5m.pt --freeze 10 --name yolov5m_frozen


## 10. Model Comparison & Conclusion

| Model | Accuracy | Speed | Stability |
|------|----------|-------|-----------|
| YOLOv5s | Medium | Fast | Good |
| YOLOv5m | High | Slower | Moderate |
| YOLOv5m Frozen | High | Faster | Best |

**Conclusion:**  
Frozen YOLOv5m provides the best balance between accuracy, generalization, and stability for anomaly detection.
